In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def rk4(x_init: np.ndarray, t_init: float, dt: float, f):
   '''
   Integrates the dynamics of `x_dot = f(x, t)` over a time interval `dt` given an initial state, `x_init`, and time, `t_init`
   using the RK4 algorithm.
   Returns the state at the end of the time interval `dt`.

   - x_init = Initial system state
   - t_init = Initial time
   - dt = Time interval over which `f` is integrated
   - f = f(t, x) - the function representing the time derivative of the state vector, `x`
   '''
   k1 = f(t_init, x_init)
   k2 = f(t_init + dt/2, x_init + dt * k1/2)
   k3 = f(t_init + dt/2, x_init + dt * k2/2)
   k4 = f(t_init + dt, x_init + dt * k3)

   return x_init + (dt / 6) * (k1 + 2 * k2 + 2 * k3 + k4)


In [3]:
def z_dot_dubins(z: np.ndarray, u: np.ndarray):
    '''
    z : [x, y, theta]
        x_pos, y_pos, orientation
        State of dynamical system
    u : [v, w]
        commanded speed, commanded angular velocity
        Control input for dynamical system
    Returns z_dot = f(z, u)
    '''
    z_dot = [u[0] * np.cos(z[2]), u[0] * np.sin(z[2]), u[1]]
    return np.array(z_dot)

In [8]:
def control_dubins(z: np.ndarray, gamma: float, zeta: float, h: float, k: float):
    '''
    z : [x, y, theta]
        x_pos, y_pos, orientation
        State of dynamical system
    z_target: [x, y, theta]
        Desired state of dynamical system
    gamma: exponential stabilization parameter > 0
    Assumes stabilizing to the origin with heading = 0
    '''
    p = np.sqrt(z[0]**2 + z[1]**2)
    theta = np.atan2(z[1], z[0])
    alpha = theta - z[2]
    u = -gamma * p * np.cos(alpha)
    w = -k * alpha + gamma * (np.sin(alpha) * np.cos(alpha) / alpha) * (alpha + h * theta)
    return np.array([u, w])

In [9]:
def make_regulated_dubins(gamma: float, zeta: float, h: float, k: float):
    f = lambda t, z: z_dot_dubins(z, control_dubins(z, gamma, zeta, h, k))
    return f

In [12]:
gamma = 5.0
zeta = 1.0
h = 1.0
k = 1.0
dt = 0.01
states = []
inputs = []
z_init = np.array([-10, 10, np.pi/4])
states.append(z_init)
f = make_regulated_dubins(gamma, zeta, h, k)
t = 0.0

for i in range(1000):
    z = rk4(states[-1], t, dt, f)
    inputs.append(control_dubins(gamma, zeta, h, k))
    t += dt
    # slacks.append(slack_dubins(states[-1], np.zeros(3), A, 2.0, 1000.))
    states.append(z)

TypeError: control_dubins() missing 1 required positional argument: 'k'